In [1]:
import pickle
import spacy
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB, CategoricalNB
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [2]:
nlp = spacy.load("en_core_web_sm", disable=["parser", "textcat", "ner"])

In [3]:
df = pd.read_csv('winemag-data-130k-v2.csv', index_col = 0)

In [5]:
df.columns

Index(['country', 'description', 'designation', 'points', 'price', 'province',
       'region_1', 'region_2', 'taster_name', 'taster_twitter_handle', 'title',
       'variety', 'winery'],
      dtype='object')

In [6]:
df.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [7]:
def custom_tokenizer(text):
    '''
    used to filter out unwanted words, punctuation, and so on
    '''
    tokens = []
    for t in nlp(text):
        if not(len(t) < 2 or t.is_stop or t.like_num or 
               t.is_punct or not t.is_alpha):
            tokens.append(t.lemma_)
    return tokens 

In [8]:
corpus = df['description']

In [9]:
corpus

0         Aromas include tropical fruit, broom, brimston...
1         This is ripe and fruity, a wine that is smooth...
2         Tart and snappy, the flavors of lime flesh and...
3         Pineapple rind, lemon pith and orange blossom ...
4         Much like the regular bottling from 2012, this...
                                ...                        
129966    Notes of honeysuckle and cantaloupe sweeten th...
129967    Citation is given as much as a decade of bottl...
129968    Well-drained gravel soil gives this wine its c...
129969    A dry style of Pinot Gris, this is crisp with ...
129970    Big, rich and off-dry, this is powered by inte...
Name: description, Length: 129971, dtype: object

In [10]:
y = df['variety']

In [11]:
y.value_counts()

Pinot Noir                  13272
Chardonnay                  11753
Cabernet Sauvignon           9472
Red Blend                    8946
Bordeaux-style Red Blend     6915
                            ...  
St. Vincent                     1
Ramisco                         1
Tinta Madeira                   1
Centesimino                     1
Dafni                           1
Name: variety, Length: 707, dtype: int64

In [12]:
bow = CountVectorizer(tokenizer=custom_tokenizer, 
                      ngram_range=(1, 1), 
                      min_df=0.01, 
                      max_df=0.99)

In [13]:
corpus_train, corpus_test, y_train, y_test = train_test_split(corpus, y, 
                                                              test_size=0.4, 
                                                              train_size=0.6)

In [14]:
X_train = bow.fit_transform(corpus_train)

In [15]:
X_train.shape

(77982, 395)

In [16]:
X_test = bow.transform(corpus_test)

In [17]:
m_nb = Pipeline([
    ('TfIdf', TfidfTransformer()),
    ('NB', BernoulliNB())
])

In [18]:
cross_val_score(m_nb, X_train, y_train, scoring='accuracy', n_jobs=4, cv=4).mean()

TypeError: '<' not supported between instances of 'float' and 'str'

In [ ]:
m_nb.fit(X_train, y_train)

In [ ]:
m_nb.predict(X_test)

In [ ]:
#metrics.accuracy_score(y_test, m_nb.predict(X_test))

In [ ]:
# get the NaiveBayes form the pipeline and return marginal probabilites P(X|Y=k)
feat_imp = m_nb['NB'].feature_log_prob_
feat_imp.shape

In [ ]:
feat_imp = pd.DataFrame(feat_imp, columns=bow.get_feature_names()).T
np.exp(feat_imp)

In [ ]:
m_nb.predict_proba(bow.transform(['cherry flavor nice oak accent'])).max()

In [ ]:
m_nbg = Pipeline([
    ('TfIdf', TfidfTransformer()),
    ('NB', GaussianNB())
])

In [ ]:
cross_val_score(m_nbg, X_train, y_train, scoring='accuracy', n_jobs=4, cv=4).mean()

In [ ]:
m_nbm = Pipeline([
    ('TfIdf', TfidfTransformer()),
    ('NB', MultinomialNB())
])

In [ ]:
cross_val_score(m_nbm, X_train, y_train, scoring='accuracy', n_jobs=4, cv=4).mean()

In [31]:
m_nbc = Pipeline([
    ('TfIdf', TfidfTransformer()),
    ('NB', CategoricalNB())
])

In [32]:
cross_val_score(m_nbc, X_train, y_train, scoring='accuracy', n_jobs=4, cv=4).mean()

/home/samuelmcguire/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  % (min_groups, self.n_splits)), UserWarning)


nan

In [33]:
m_rf = Pipeline([
    ('TfIdf', TfidfTransformer()),
    ('RFC', RandomForestClassifier())
])

In [ ]:
cross_val_score(m_rf, X_train, y_train, scoring='accuracy',  cv=4).mean()

/home/samuelmcguire/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  % (min_groups, self.n_splits)), UserWarning)


In [222]:
m_rfr = Pipeline([
    ('TfIdf', TfidfTransformer()),
    ('RFC', RandomForestRegressor())
])

In [223]:
cross_val_score(m_rfr, X_train, y_train, scoring='accuracy', n_jobs=4, cv=4).mean()

nan

In [ ]:
m = Pipeline([
    ('TfIdf', TfidfTransformer()),
    ('LogReg', LogisticRegression(class_weight='balanced'))
    ])

In [ ]:
cross_val_score(m, X_train, y_train, scoring='accuracy', n_jobs=4, cv=4).mean()